# Affective Dimensions of UI Training: Statistical Analysis of Results 

In [3]:
import pandas as pd 

In [5]:
NO_GUIDE_IMI = pd.read_csv('./no-guide-all-questionnaires.csv')
NO_GUIDE_IMI['condition'] = 'no-guide'
SIMON_IMI = pd.read_csv('./simon-all-questionnaires.csv')
SIMON_IMI['condition'] = 'simon'
SPOTLIGHT_IMI = pd.read_csv('./spotlight-all-questionnaires.csv')
SPOTLIGHT_IMI['condition'] = 'spotlight'

IMI = pd.concat([NO_GUIDE_IMI, SIMON_IMI, SPOTLIGHT_IMI], ignore_index=True)

In [25]:
# Note Side Effects: This function will alter the original dataframe (shallow copy)
def format_likert_data_for_stats(df: pd.DataFrame, likert_columns: list) -> pd.DataFrame:
    for column_name in likert_columns:
        df[column_name] = df[column_name].replace('1 (Strongly Disagree)', '1')
        df[column_name] = df[column_name].replace('7 (Strongly Agree)', '7')
        df[column_name] = df[column_name].replace('4 (Neutral)', '4')
        df[column_name] = df[column_name].astype('int64')
    return df

In [20]:

imi_dimensions = ['already_memorized',
       'dont_want_to_continue', 'boring_task', 'satisfied_performance',
       'attention', 'effort_task', 'relaxed_task', 'help_memorize_commands']

## ART-transformed ANOVA 

In [11]:
! pip install rpy2

     |████████████████████████████████| 79 kB 1.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  ERROR: Command errored out with exit status 1:
   command: /home/ashley/anaconda3/bin/python /home/ashley/anaconda3/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py get_requires_for_build_wheel /tmp/tmpj9qbjihj
       cwd: /tmp/pip-install-iuzun3zl/rpy2-rinterface_64d9fbffb7374316b6dac2318f7bfa26
  Complete output (52 lines):
  /home/ashley/anaconda3/compiler_compat/ld: /lib/x86_64-linux-gnu/libm.so.6: unknown type [0x13] section `.relr.dyn'
  /home/ashley/anaconda3/compiler_compat/ld: skipping incompatible /lib/x86_64-linux-gnu/libm.so.6 when searching for /lib/x86_64-linux-gnu/libm.so.6
  /home/ashley/anaconda3/compiler_compat/ld: cannot find /lib/x86_64-linux-gnu/libm.so.6
  /home/ashley/anaconda3/compiler_compat/ld: /lib/x86_64-linux-gnu/libm.so.6: unknown type [0x13] section `.relr.dyn'
  /home/ashley/

In [ ]:
! sudo apt-get install libstdc++6


SyntaxError: invalid syntax (3292570957.py, line 2)

In [ ]:
from statsmodels.formula.api import ols
from rpy2.robjects import pandas2ri
import statsmodels.api as sm
import rpy2.robjects as ro


pandas2ri.activate()
ro.r('library(ARTool)')

art_results = {}
for dim in imi_dimensions:
    # Prepare data for R
    df_r = IMI[['condition', dim]].copy()
    df_r[dim] = df_r[dim].astype(str)
    df_r = df_r.dropna()
    pandas2ri.activate()
    ro.globalenv['df_r'] = df_r
    ro.r(f'art_model <- art({dim} ~ condition, data = df_r)')
    ro.r('anova_res <- anova(art_model)')
    art_results[dim] = ro.r('anova_res')

    # Example: print ART ANOVA result for one dimension
print(art_results['attention'])


OSError: cannot load library '/usr/lib/R/lib/libR.so': /home/ashley/anaconda3/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /lib/x86_64-linux-gnu/libicuuc.so.74)

## Kruskal Wallis test 

In [ ]:
import pandas
from scipy.stats import mstats

SIMON_IMI = format_likert_data_for_stats(SIMON_IMI, imi_dimensions)
SPOTLIGHT_IMI = format_likert_data_for_stats(SPOTLIGHT_IMI, imi_dimensions)
NO_GUIDE_IMI = format_likert_data_for_stats(NO_GUIDE_IMI, imi_dimensions)

print("Kruskal Wallis H-test test:")

for dimension in imi_dimensions:
    H, pval = mstats.kruskalwallis(SIMON_IMI[dimension].values, SPOTLIGHT_IMI[dimension].values, NO_GUIDE_IMI[dimension].values)
    print(f"Measuring {dimension}...")
    print("H-statistic:", H)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

Kruskal Wallis H-test test:
Measuring already_memorized...
H-statistic: 5.100122800996366e+01
P-Value: 8.418293293266031e-12
Reject NULL hypothesis - Significant differences exist between groups.
Measuring dont_want_to_continue...
H-statistic: 9.051596706312921
P-Value: 0.010826067959896643
Reject NULL hypothesis - Significant differences exist between groups.
Measuring boring_task...
H-statistic: 15.52498886580413
P-Value: 0.00042539413900512474
Reject NULL hypothesis - Significant differences exist between groups.
Measuring satisfied_performance...
H-statistic: 92.73750949778803
P-Value: 7.28292235449495e-21
Reject NULL hypothesis - Significant differences exist between groups.
Measuring attention...
H-statistic: 1.5882352941144564
P-Value: 0.45197986479438423
Accept NULL hypothesis - No significant difference between groups.
Measuring effort_task...
H-statistic: 58.71921499353609
P-Value: 1.7753519982758068e-13
Reject NULL hypothesis - Significant differences exist between groups.
M

In [27]:
for dimension in imi_dimensions:
    H, pval = mstats.kruskalwallis(SIMON_IMI[dimension].values, SPOTLIGHT_IMI[dimension].values)
    print(f"Measuring {dimension}...")
    print("H-statistic:", H)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

Measuring already_memorized...
H-statistic: 44.05242762917112
P-Value: 3.1969712304892714e-11
Reject NULL hypothesis - Significant differences exist between groups.
Measuring dont_want_to_continue...
H-statistic: 1.6188744488628732
P-Value: 0.20324880105460447
Accept NULL hypothesis - No significant difference between groups.
Measuring boring_task...
H-statistic: 12.0936767893434
P-Value: 0.0005059310698957161
Reject NULL hypothesis - Significant differences exist between groups.
Measuring satisfied_performance...
H-statistic: 81.96252563324092
P-Value: 1.3869156641918706e-19
Reject NULL hypothesis - Significant differences exist between groups.
Measuring attention...
H-statistic: 0.7843137254863722
P-Value: 0.375825087489862
Accept NULL hypothesis - No significant difference between groups.
Measuring effort_task...
H-statistic: 12.18424820353688
P-Value: 0.0004819475901306014
Reject NULL hypothesis - Significant differences exist between groups.
Measuring relaxed_task...
H-statistic: 

In [28]:
for dimension in imi_dimensions:
    H, pval = mstats.kruskalwallis(SPOTLIGHT_IMI[dimension].values, NO_GUIDE_IMI[dimension].values)
    print(f"Measuring {dimension}...")
    print("H-statistic:", H)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

Measuring already_memorized...
H-statistic: 0.39837577162450766
P-Value: 0.5279292702983308
Accept NULL hypothesis - No significant difference between groups.
Measuring dont_want_to_continue...
H-statistic: 2.6847163498624047
P-Value: 0.10131526514675834
Accept NULL hypothesis - No significant difference between groups.
Measuring boring_task...
H-statistic: 0.32231132711413324
P-Value: 0.5702219217195807
Accept NULL hypothesis - No significant difference between groups.
Measuring satisfied_performance...
H-statistic: 3.542180677398545
P-Value: 0.05982675750629062
Accept NULL hypothesis - No significant difference between groups.


ValueError: All numbers are identical in kruskal

## Mann-Whitney

In [29]:
import scipy.stats as stats

for dimension in imi_dimensions:
    statistic, pval = stats.mannwhitneyu(SIMON_IMI[dimension], SPOTLIGHT_IMI[dimension], alternative='two-sided')
    print(f"Measuring {dimension}...")
    print("statistic:", statistic)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

#perform the Mann-Whitney U test




Measuring already_memorized...
statistic: 5059.5
P-Value: 3.214479498601805e-11
Reject NULL hypothesis - Significant differences exist between groups.
Measuring dont_want_to_continue...
statistic: 8387.0
P-Value: 0.20353385756431974
Accept NULL hypothesis - No significant difference between groups.
Measuring boring_task...
statistic: 11346.5
P-Value: 0.0005074478693700791
Reject NULL hypothesis - Significant differences exist between groups.
Measuring satisfied_performance...
statistic: 3427.5
P-Value: 1.3969496307575646e-19
Reject NULL hypothesis - Significant differences exist between groups.
Measuring attention...
statistic: 9120.0
P-Value: 0.3798163312134514
Accept NULL hypothesis - No significant difference between groups.
Measuring effort_task...
statistic: 7071.5
P-Value: 0.0004834427493986909
Reject NULL hypothesis - Significant differences exist between groups.
Measuring relaxed_task...
statistic: 4560.0
P-Value: 3.1345902826796593e-13
Reject NULL hypothesis - Significant diff

In [30]:

for dimension in imi_dimensions:
    statistic, pval = stats.mannwhitneyu(SIMON_IMI[dimension], NO_GUIDE_IMI[dimension], alternative='two-sided')
    print(f"Measuring {dimension}...")
    print("statistic:", statistic)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

Measuring already_memorized...
statistic: 5977.0
P-Value: 3.6079506093317154e-08
Reject NULL hypothesis - Significant differences exist between groups.
Measuring dont_want_to_continue...
statistic: 7504.0
P-Value: 0.0026803841594193164
Reject NULL hypothesis - Significant differences exist between groups.
Measuring boring_task...
statistic: 11382.0
P-Value: 0.002003071500435297
Reject NULL hypothesis - Significant differences exist between groups.
Measuring satisfied_performance...
statistic: 4925.0
P-Value: 4.286089835379906e-12
Reject NULL hypothesis - Significant differences exist between groups.
Measuring attention...
statistic: 9348.0
P-Value: 0.37382727484188627
Accept NULL hypothesis - No significant difference between groups.
Measuring effort_task...
statistic: 4685.5
P-Value: 2.6232101451014284e-15
Reject NULL hypothesis - Significant differences exist between groups.
Measuring relaxed_task...
statistic: 5706.0
P-Value: 1.1555972596799649e-08
Reject NULL hypothesis - Significa

In [31]:

for dimension in imi_dimensions:
    statistic, pval = stats.mannwhitneyu(NO_GUIDE_IMI[dimension], SPOTLIGHT_IMI[dimension], alternative='two-sided')
    print(f"Measuring {dimension}...")
    print("statistic:", statistic)
    print("P-Value:", pval)

    if pval < 0.05:
        print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
        print("Accept NULL hypothesis - No significant difference between groups.")

Measuring already_memorized...
statistic: 7046.5
P-Value: 0.5285481184851544
Accept NULL hypothesis - No significant difference between groups.
Measuring dont_want_to_continue...
statistic: 8241.5
P-Value: 0.10151362947194248
Accept NULL hypothesis - No significant difference between groups.
Measuring boring_task...
statistic: 7664.5
P-Value: 0.5708997184992899
Accept NULL hypothesis - No significant difference between groups.
Measuring satisfied_performance...
statistic: 6404.0
P-Value: 0.05995777065326226
Accept NULL hypothesis - No significant difference between groups.
Measuring attention...
statistic: 7380.0
P-Value: 1.0
Accept NULL hypothesis - No significant difference between groups.
Measuring effort_task...
statistic: 8751.5
P-Value: 0.0012122559825603813
Reject NULL hypothesis - Significant differences exist between groups.
Measuring relaxed_task...
statistic: 6158.0
P-Value: 0.01974227910823589
Reject NULL hypothesis - Significant differences exist between groups.
Measuring 